In [106]:
from tools import get_puzzle, show_problem_1
TODAY = 12
puzzle = get_puzzle(TODAY)
show_problem_1(puzzle)

<!DOCTYPE html>
<html lang="en-us">
<head>
<meta charset="utf-8"/>
<title>Day 12 - Advent of Code 2022</title>
<!--[if lt IE 9]><script src="/static/html5.js"></script><![endif]-->
<link href='//fonts.googleapis.com/css?family=Source+Code+Pro:300&subset=latin,latin-ext' rel='stylesheet' type='text/css'/>
<link rel="stylesheet" type="text/css" href="/static/style.css?30"/>
<link rel="stylesheet alternate" type="text/css" href="/static/highcontrast.css?0" title="High Contrast"/>
<link rel="shortcut icon" href="/favicon.png"/>
<script>window.addEventListener('click', function(e,s,r){if(e.target.nodeName==='CODE'&&e.detail===3){s=window.getSelection();s.removeAllRanges();r=document.createRange();r.selectNodeContents(e.target);s.addRange(r);}});</script>
</head><!--




Oh, hello!  Funny seeing you here.

I appreciate your enthusiasm, but you aren't going to find much down here.
There certainly aren't clues to any of the puzzles.  The best surprises don't
even appear in the source until you

## --- Day 12: Hill Climbing Algorithm ---


You try contacting the Elves using yourhandheld device, but the river you're following must be too low to get a decent signal.


You ask the device for a heightmap of the surrounding area (your puzzle input). The heightmap shows the local area from above broken into a grid; the elevation of each square of the grid is given by a single lowercase letter, wherea``is the lowest elevation,b``is the next-lowest, and so on up to the highest elevation,z``.


Also included on the heightmap are marks for your current position (S``) and the location that should get the best signal (E``). Your current position (S``) has elevationa``, and the location that should get the best signal (E``) has elevationz``.


You'd like to reachE``, but to save energy, you should do it in **as few steps as possible** . During each step, you can move exactly one square up, down, left, or right. To avoid needing to get out your climbing gear, the elevation of the destination square can be **at most one higher** than the elevation of your current square; that is, if your current elevation ism``, you could step to elevationn``, but not to elevationo``. (This also means that the elevation of the destination square can be much lower than the elevation of your current square.)


For example:


SE```
 abqponm
 abcryxxl
 accszxk
 acctuvwj
 abdefghi

```


Here, you start in the top-left corner; your goal is near the middle. You could start by moving down or right, but eventually you'll need to head toward thee``at the bottom. From there, you can spiral around to the goal:


```
 v..v<<<<
 >v.vv<<^
 .>vv>E^^
 ..v>>>^^
 ..>>>>>^

```


In the above diagram, the symbols indicate whether the path exits each square moving up (^``), down (v``), left (<``), or right (>``). The location that should get the best signal is stillE``, and.``marks unvisited squares.


This path reaches the goal in31``steps, the fewest possible.


 **What is the fewest steps required to move from your current position to the location that should get the best signal?** 




In [300]:
from collections import deque
dbg = False

def find_start_end(data):
    for i, row in enumerate(data):
        for j,letter in enumerate (row):
            if letter == 'S':
                start = (i,j)
            if letter == 'E':
                end = (i,j)
    return start, end

def get_next_steps(curr_pos, curr_steps, data, steps_matrix):
    for dirx,diry in [ (1,0),(0,1),(-1,0),(0,-1) ]:
        x,y = curr_pos
        _next = (x+dirx, y+diry)
        if len(data) > (x+dirx) >= 0 and  len(data[0]) > y+diry >= 0: # check not going out of bounds

            if  curr_steps+1 < steps_matrix[x+dirx][y+diry]:  #this path is shorter than previously calculated

                curr_letter = data[x][y]
                if curr_letter == 'S':
                    curr_letter = 'a'
                next_letter = data[x+dirx][y+diry]
                if next_letter == 'E':
                    next_letter = 'z'
                if ord(next_letter) - ord(curr_letter) <= 1 : #step is permitted
                    steps_matrix [x+dirx][y+diry] = curr_steps +1
                    yield ((x+dirx, y+diry), curr_steps +1)


DATA = puzzle.data

steps_matrix = [[41*137 for __ in _] for _ in DATA  ] # 41*137 is the size of the data matrix. no step number can/should be bigger than this

start, end  = find_start_end(DATA)

steps_matrix[start[0]][start[1]] = 1

to_visit = deque()
to_visit.append((start,1))

while to_visit:
    curr_pos,curr_steps = to_visit.popleft()
    x,y = curr_pos
    for _next in get_next_steps(curr_pos, curr_steps, DATA, steps_matrix):
        to_visit.append(_next)
    
print( f"Solution 1: minimum steps needed {steps_matrix[end[0]][end[1]] -1}")

Solution 1: minimum steps needed 456


In [140]:
from tools import show_problem_2
show_problem_2(puzzle)


## --- Part Two ---


As you walk up the hill, you suspect that the Elves will want to turn this into a hiking trail. The beginning isn't very scenic, though; perhaps you can find a better starting point.


To maximize exercise while hiking, the trail should start as low as possible: elevationa``. The goal is still the square markedE``. However, the trail should still be direct, taking the fewest steps to reach its goal. So, you'll need to find the shortest path from **any square at elevationa``** to the square markedE``.


Again consider the example from above:


SE```
 abqponm
 abcryxxl
 accszxk
 acctuvwj
 abdefghi

```


Now, there are six choices for starting position (five markeda``, plus the square markedS``that counts as being at elevationa``). If you start at the bottom-left square, you can reach the goal most quickly:


```
 ...v<<<<
 ...vv<<^
 ...v>E^^
 .>v>>>^^
 >^>>>>>^

```


This path reaches the goal in only29``steps, the fewest possible.


 **What is the fewest steps required to move starting from any square with elevationa``to the location that should get the best signal?** 




In [310]:
from collections import deque
dbg = False

def find_highest_a(data, steps_matrix):
    highest_a = (9999, (0,0))
    for i,row in enumerate(data):
        for j,_char in enumerate(row):
            if _char == 'a':
                #print(f"{i}, {j}")
                height_to_compare = steps_matrix[i][j]
                #print(f"highest_a [0] {highest_a [0]}    height_to_compare {height_to_compare}")
                if highest_a [0] > height_to_compare:
                    highest_a = (height_to_compare, (i,j) )
                

    return highest_a

def find_end(data):

    for i, row in enumerate(data):
        for j,letter in enumerate (row):
            if letter == 'E':
                return (i,j)


def get_next_steps(curr_pos, curr_steps, data, steps_matrix):
    for dirx,diry in [ (1,0),(0,1),(-1,0),(0,-1) ]:
        x,y = curr_pos
        _next = (x+dirx, y+diry)

        if len(data) > (x+dirx) >= 0 and  len(data[0]) > y+diry >= 0: # check not going out of bounds
            if  curr_steps+1 < steps_matrix[x+dirx][y+diry]:  #this path is shorter than previously calculated
                curr_letter = data[x][y]
                if curr_letter == 'E':
                    curr_letter = 'z'
                next_letter = data[x+dirx][y+diry]
                if next_letter == 'S':
                    next_letter = 'a'

                step = ord(curr_letter) -ord(next_letter)

                if step < 2  : #stepping down 1 or 0 or going up as much as we want
                    steps_matrix [x+dirx][y+diry] = curr_steps +1
                    yield ((x+dirx, y+diry), curr_steps +1)

def solution():
    DATA = puzzle.data
    steps_matrix = [[41*137 for __ in _] for _ in DATA  ] # 41*137 is the size of the data matrix. no step number can/should be bigger than this

    start, end  = find_start_end(DATA)
    # since we are walking backwards (down the mountain) we set the end as 1
    steps_matrix[end[0]][end[1]] = 1
    #and add it as the first place to visit
    to_visit = deque()
    to_visit.append((end,1))
    while to_visit:
        curr_pos,curr_steps = to_visit.popleft()
        x,y = curr_pos
        for _next in get_next_steps(curr_pos, curr_steps, DATA, steps_matrix):
            to_visit.append(_next)
    print( f"Solution 2: minimum steps needed {find_highest_a(DATA, steps_matrix)}")
    
solution ()

In [311]:
import timeit
timeit.timeit(solution,number=100)

0.7174229619995458